# Câu 1:

In [10]:
import sqlite3
import math

conn = sqlite3.connect(":memory:")

# Đăng ký hàm
conn.create_function("SQRT", 1, math.sqrt)
conn.create_function("POWER", 2, math.pow)



In [11]:
import sqlite3
import pandas as pd
import math

# Kết nối và tạo hàm
conn = sqlite3.connect(":memory:")
conn.create_function("SQRT", 1, math.sqrt)
conn.create_function("POWER", 2, math.pow)

# Tạo bảng mẫu
df = pd.DataFrame({'A': [1, 2, 3, 4], 'B': [2, 4, 6, 8]})
df.to_sql('your_table', conn, index=False, if_exists='replace')

# Câu SQL
query = """
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - POWER(SUM(A), 2)) *
     SQRT(COUNT(*) * SUM(B * B) - POWER(SUM(B), 2))) AS r_AB
FROM your_table;
"""

# Chạy truy vấn
result = pd.read_sql_query(query, conn)
print(result)


   r_AB
0   1.0


# Câu 2: 

In [12]:
import pandas as pd
from scipy.stats import chi2_contingency

# Tạo dữ liệu dạng rộng
data = {
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
}
df = pd.DataFrame(data)

# Chuyển sang dạng long format
df_long = df.melt(id_vars='Day', var_name='Model', value_name='Score')

# Phân loại điểm thành Low / Medium / High
def categorize(score):
    if score <= 6:
        return 'Low'
    elif score < 8:
        return 'Medium'
    else:
        return 'High'

df_long['Category'] = df_long['Score'].apply(categorize)

# Kiểm định Chi-Square theo Model
table_model = pd.crosstab(df_long['Model'], df_long['Category'])
chi2_model, p_model, _, _ = chi2_contingency(table_model)

# Kiểm định Chi-Square theo Day
table_day = pd.crosstab(df_long['Day'], df_long['Category'])
chi2_day, p_day, _, _ = chi2_contingency(table_day)

# In kết quả
print("Bảng chéo theo Model:\n", table_model)
print(f"Chi2 theo Model = {chi2_model:.4f}, p = {p_model:.4f}\n")

print("Bảng chéo theo Day:\n", table_day)
print(f"Chi2 theo Day = {chi2_day:.4f}, p = {p_day:.4f}")

Bảng chéo theo Model:
 Category  High  Low  Medium
Model                      
A            1    1       2
B            2    1       1
C            1    1       2
Chi2 theo Model = 0.9000, p = 0.9246

Bảng chéo theo Day:
 Category  High  Low  Medium
Day                        
Day 1        2    0       1
Day 2        1    0       2
Day 3        1    1       1
Day 4        0    2       1
Chi2 theo Day = 6.2667, p = 0.3940


# Câu 3:

In [14]:
import sqlite3
import random

# Kết nối đến SQLite
conn = sqlite3.connect("flights.db")
cursor = conn.cursor()

# Tạo bảng chỉ chứa departure_time_int
cursor.execute("""
CREATE TABLE IF NOT EXISTS flights (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    departure_time_int INTEGER
)
""")

# Xoá dữ liệu cũ
cursor.execute("DELETE FROM flights")

# Hàm tạo thời gian ngẫu nhiên
def generate_time():
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    return hour * 100 + minute

# Sinh và chèn dữ liệu mới
departure_times = [(generate_time(),) for _ in range(100)]
cursor.executemany("INSERT INTO flights (departure_time_int) VALUES (?)", departure_times)
conn.commit()

# Truy vấn SQL: chuyển int → thời gian chuỗi 'HH:MM'
cursor.execute("""
SELECT 
    id,
    departure_time_int,
    printf('%02d:%02d', departure_time_int / 100, departure_time_int % 100) AS formatted_time
FROM flights
LIMIT 50
""")
rows = cursor.fetchall()

# In kết quả
print("ID | Departure Time (Int) | Departure Time (Formatted)")
print("-" * 50)
for row in rows:
    print(f"{row[0]:<3} | {row[1]:<20} | {row[2]}")

conn.close()


ID | Departure Time (Int) | Departure Time (Formatted)
--------------------------------------------------
101 | 13                   | 00:13
102 | 1337                 | 13:37
103 | 1425                 | 14:25
104 | 1235                 | 12:35
105 | 2309                 | 23:09
106 | 841                  | 08:41
107 | 1238                 | 12:38
108 | 632                  | 06:32
109 | 2333                 | 23:33
110 | 453                  | 04:53
111 | 313                  | 03:13
112 | 700                  | 07:00
113 | 236                  | 02:36
114 | 2142                 | 21:42
115 | 1756                 | 17:56
116 | 1117                 | 11:17
117 | 121                  | 01:21
118 | 419                  | 04:19
119 | 139                  | 01:39
120 | 2159                 | 21:59
121 | 2112                 | 21:12
122 | 113                  | 01:13
123 | 516                  | 05:16
124 | 1517                 | 15:17
125 | 912                  | 09:12
126 | 1005         

# Câu 4:

In [15]:
import sqlite3
import random
import pandas as pd
import numpy as np

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng
cursor.execute('''
CREATE TABLE data (
    id INTEGER PRIMARY KEY,
    value REAL
);
''')

# Thêm dữ liệu ngẫu nhiên
for i in range(150):
    value = round(random.uniform(0, 80), 2)
    cursor.execute('INSERT INTO data (id, value) VALUES (?, ?)', (i + 1, value))
conn.commit()

# Đọc dữ liệu vào DataFrame
df = pd.read_sql_query("SELECT * FROM data", conn)

# Tính median và MAD bằng pandas
median = df['value'].median()
mad = (df['value'] - median).abs().median()

# Tìm ngoại lệ
df['is_outlier'] = (df['value'] - median).abs() > 1.5 * mad

# Phân chia
outliers = df[df['is_outlier']]
non_outliers = df[~df['is_outlier']]

# Gộp lại để hiển thị
result = pd.merge(
    outliers[['id', 'value']].rename(columns={'value': 'Ngoại lệ'}),
    non_outliers[['id', 'value']].rename(columns={'value': 'Không phải ngoại lệ'}),
    left_index=True, right_index=True, how='outer'
)

# Hiển thị
pd.set_option('display.max_rows', None)
print(f"MAD = {mad}")
print(result)

conn.close()


MAD = 19.11
      id_x  Ngoại lệ   id_y  Không phải ngoại lệ
0      1.0     77.91    NaN                  NaN
1      2.0     79.64    NaN                  NaN
2      3.0     69.27    NaN                  NaN
3      NaN       NaN    4.0                55.86
4      NaN       NaN    5.0                22.74
5      6.0     68.48    NaN                  NaN
6      NaN       NaN    7.0                51.96
7      NaN       NaN    8.0                16.39
8      NaN       NaN    9.0                25.00
9     10.0     72.45    NaN                  NaN
10     NaN       NaN   11.0                10.57
11    12.0      7.45    NaN                  NaN
12    13.0      6.49    NaN                  NaN
13     NaN       NaN   14.0                38.39
14     NaN       NaN   15.0                28.58
15    16.0      8.31    NaN                  NaN
16    17.0     77.08    NaN                  NaN
17     NaN       NaN   18.0                61.80
18    19.0     76.33    NaN                  NaN
19     N

# Câu 5:

In [16]:
import sqlite3

# Tạo kết nối
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
);
''')

# Thêm dữ liệu mẫu
data = [
    ('Nguyen', 60.0, 165),
    ('Nguyen', 61.5, 164),
    ('Tran', 75.0, 170),
    ('Le', 49.0, 160),
    ('Le', 70.0, 180)
]
cursor.executemany('INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)', data)

# Tìm các cặp người có thể là cùng một người dựa trên Boolean:
# - last_name giống nhau
# - |weight1 - weight2| <= 2
cursor.execute('''
SELECT 
    p1.id AS id1, p2.id AS id2,
    p1.last_name, p1.weight, p2.weight,
    ABS(p1.weight - p2.weight) AS weight_diff
FROM 
    Patient p1
JOIN 
    Patient p2 
ON 
    p1.id < p2.id
WHERE 
    p1.last_name = p2.last_name
    AND ABS(p1.weight - p2.weight) <= 2;
''')

rows = cursor.fetchall()

# In kết quả
print("Các cặp bệnh nhân có thể là cùng một người (dựa trên last_name và weight gần giống):")
for row in rows:
    print(f"ID {row[0]} & ID {row[1]} | Họ: {row[2]} | Cân nặng: {row[3]} - {row[4]} | Chênh lệch: {row[5]} kg")

conn.close()


Các cặp bệnh nhân có thể là cùng một người (dựa trên last_name và weight gần giống):
ID 1 & ID 2 | Họ: Nguyen | Cân nặng: 60.0 - 61.5 | Chênh lệch: 1.5 kg
